In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter


In [2]:
###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

def initialize_database():
    #establish connection to mongodb and fallbacks
    CONNECTION_CREDENTIALS = [
        {
            'username': "hmarx0253",
            'password': "opal98z",
            #added additional required parameters
            'host': "nv-desktop-services.apporto.com",
            'port': 31084,
            'database': "AAC",
            'collection': "animals"
        },
        #localhost fallback
        {
            'username': "",
            'password': "",
            'host': "localhost",
            'port': 27017,
            'database': "AAC",
            'collection': "animals"
        }
    ]
    
    for creds in CONNECTION_CREDENTIALS:
        try:
            print(f"trying connection to {creds['host']}:{creds['port']}.....")
            # Connect to database via CRUD Module
            db = AnimalShelter(
                creds['username'],
                creds['password'],
                creds['host'],
                creds['port'],
                creds['database'],
                creds['collection']
            )
            #check and validate connection
            if db.read({}, limit=1):
                print("connection was established")
                return db
        except Exception as e:
            print(f"cannot establish connection to {creds['host']}: {str(e)}")
            
    print("MONGODB CONNECTIONS FAILED. SAMPLE DATA WILL BE USED")
    return None
    
def load_animal_data(db_connection):
    #load error handling for data
    sample_data = {
        'name' : ['Buddy', 'Max', 'Bella'],
        'breed': ['Labrador Retriever', 'German Shepherd', 'Siberian Husky'],
        'age_upon_outcome': ['2 years', '1 year', '3 years'],
        'sex_upon_outcome': ['Neutered Male', 'Intact Male', 'Spayed Female'],
        'location_lat': [30.26, 30.28, 30.25],
        'location_long': [-97.74, -97.72, -97.76]
    }
    
    try:
        if db_connection:
            records = db_connection.read({})
            if records:
                df = pd.DataFrame.from_records(records)
                if '_id' in df.columns:
                    df.drop(columns=['_id'],inplace=True)
                return df
        return pd.DataFrame(sample_data)
    except Exception as e:
        print(f"error loading data: {str(e)}")
        return pd.DataFrame(sample_data)

#initialize database and load data 
db = initialize_database()
df = load_animal_data(db)

trying connection to nv-desktop-services.apporto.com:31084.....
connected to MongoDB
cannot establish connection to nv-desktop-services.apporto.com: read() got an unexpected keyword argument 'limit'
trying connection to localhost:27017.....
cannot establish connection to localhost: The empty string is not valid username.
MONGODB CONNECTIONS FAILED. SAMPLE DATA WILL BE USED


In [3]:

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('ascii')

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
app.layout = html.Div([
    html.Div([
        html.A( #encodes hyperlink
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image),
                style={'height' : '100px', 'float':'left'}
            ),
        
            href='https://www.snhu.edu',
            target = '_blank'
        ),
        html.H1(
            'CS340 Data Dashboard - Grazioso Salvare Animal Rescue',
            style={'text-align' : 'center', 'color' : '#FE5E07', 'margin-top':'20px'}
        ),
        html.Div([ #identifier, name and github link
            'Developer: Hunter Marx (ketsp8ce)',
            #style = {'text-align' : 'center', 'font-style' : 'italic', 'color' : ####}
                html.A(
                    'Github',
                    href='https://github.com/ketsp8ce',
                    target='_blank',
                    style = {'color': '#3498db', 'text-decoration':'none', 'font-weight' : 'bold'}
                )
        ], style = {'text-align': 'center','font-style':'italic'})
    ]),
    
    html.Hr(),
        
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
#interactive filtering
    
    
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                         
                         
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    #filter controls
    html.Div([
        html.H3('select rescue type:', style={'text-align':'center'}),
        dcc.RadioItems(
            id = 'rescue-type-filter',
            options = [
                {'label': 'All Animals', 'value' : 'all'},
                {'label': 'Water Rescue', 'value' : 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value' : 'mountain'},
                {'label': 'Disaster/Individual Tracking', 'value' : 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value = 'all',
            labelStyle = {
                'display':'inline-block','margin':'10px','padding':'10px 20px',
                'border':'1px solid #ddd', 'border-radius':'5px', 'background-color':'#f9f9f9'
            },
            inputStyle = {'margin-right':'5px'},
            style = {'text-align':'center'} 
        )
    ], style = {'padding':'20px','background-color':'#ecf0f1', 'border-radius' : '10px'}),
    
    html.Hr(),

    #data table
    html.Div([
        dash_table.DataTable(
            id='datatable-id',
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
            data=df.to_dict('records'),
            filter_action = "native", #filter enabled
            sort_action = "native", #sort enabled
            sort_mode = "multi", #can sort multiple columns
            row_selectable = "single", #map functionality
            selected_rows = [0], #map defaults to first row
            page_action = "native", #pagination enabled
            page_current = 0, #starts at first page
            page_size = 10, #shows 10 rows per page
            style_table = {'height':'400px', 'overflowY': 'auto'}, #scroll on table enabled
            style_cell = {
                'textAlign':'left', 'minWidth':'100px', 'width':'100px', 'maxWidth':'100px', 'whiteSpace': 'normal'},
            style_header = { 'background-color': 'rgb(230,230,230)', 'font-weight': 'bold'},
            style_data_conditional = [ 
                {
                'if': {'row-index': 'odd'},
                'background-color': ('rgb(248,248,248)')
                }
            ]
        ),
    ], style={'margin':'20px'}),
    
    html.Hr(),
    
    #pie chart,
    html.Div([
        html.Div([
            html.H3('Preferred Animals',style={'text-align':'center'}),
            dcc.Graph(id='breed-pie-chart')
        ], style={'width':'48%', 'display':'inline-block', 'padding':'10px'}),
        
        html.Div([
            html.H3('Animal Location', style={'text-align':'center'}),
            html.Div(id='map-id', style={'height':'500px'})
        ], style={'width':'48%', 'display':'inline-block', 'padding':'10px'})
    ])
])

In [4]:
#############################################
# Interaction Between Components / Controller
#############################################


## FIX ME Add code to filter interactive data table with MongoDB queries


#callback to filter data based on rescue type
    
@app.callback(
    Output('datatable-id','data'),
    [Input('rescue-type-filter', 'value')]
)
def update_dashboard(filter_type):
    #filter by age and breed
    if filter_type=='water':
        query = {
            'animal_type': 'Dog',
            'breed': {'$in' : [
                'Labrador Retriever Mix',
                'Chesapeake Bay Retriever',
                'Newfoundland'
            ]},
            'sex_upon_outcome' : 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte':26, '$lte':156}
        }
    elif filter_type=='mountain':
        query = {
            'animal_type': 'Dog',
            'breed': {'$in' : [
                'German Shepherd',
                'Alaskan Malamute',
                'Old English Sheepdog',
                'Siberian Husky',
                'Rottweiler'
            ]},
            'sex_upon_outcome' : 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte':20, '$lte':300}
        }
    else: #filter all
        query = {}
    
    filtered_data = db.read(query)
    return pd.DataFrame.from_records(filtered_data).to_dict('records')
    



#
#        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
 ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData) if viewData else pd.DataFrame()
    if len(dff) == 0:
        return html.Div('no output to display')
    return dcc.Graph(
        figure = px.pie(dff, names='breed', title='Preferred Animals')
    )

#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if not viewData:
        return html.Div('select an animal to view location')
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    row = index[0] if index else 0
    
    #check existence of location columns
    if len(dff.columns) < 15:
        return html.Div('location data is not available at this time')
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75,-97.48],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                # Column 13 and 14 define the grid-coordinates for the map
                # Column 4 defines the breed for the animal
                # Column 9 defines the name of the animal
                    dl.Marker(
                        position=[dff.iloc[row,13],dff.iloc[row,14]],
                        children=[
                            dl.Tooltip(dff.iloc[row,4]),
                            dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:18270/
